# Load data

In [ ]:
import numpy as np

: 

In [ ]:
import json
import pandas as pd

# Работа с первым датасетом
with open('/home/mark/Документы/kagle/case_2_reference_without_resume_sorted.json', 'r', encoding='utf-8') as file:
    data_1 = json.load(file, )

data1 = pd.json_normalize(data_1,max_level=2)

education_cr=['Высшее', 'Бакалавр', 'Магистр'] # критерии наличия высшего образования
dataset=pd.DataFrame()
print(data1.info())
#print(type(data1['resumes'][0][1]))
education=[]
uuid=[]
average_workout=[]
employer=[]
work_city=[]
work_position=[]
work_description=[]
language=[]
about=[]
key_skills=[]
for i in range(0, len(data1['resumes'][0])):
        print('moving to i=', i)
        # Выделение датасета с инфой по работе
        uuid.append(data1['resumes'][0][i]['uuid'])
        try:
            language.append(data1['resumes'][0][i]['languageItems'])
        except:
            language.append(None)
        about.append(data1['resumes'][0][i]['about'])
        key_skills.append(data1['resumes'][0][i]['key_skills'])
        try:
            data1_conf_exp=pd.json_normalize(data1['resumes'][0][i]['experienceItem'])
            data1_conf_exp['starts']=pd.to_datetime(data1_conf_exp['starts'])
            data1_conf_exp['ends']=pd.to_datetime(data1_conf_exp['ends'])
            
            data1_conf_exp['ends'] = data1_conf_exp['ends'].fillna(pd.to_datetime("2024-02-29 00:00:00"))
            
            data1_conf_exp['duration']=(data1_conf_exp['ends']-data1_conf_exp['starts']).dt.days
            closest_date=max(data1_conf_exp['starts'])
            average_workout.append(data1_conf_exp['duration'].mean())
            line=data1_conf_exp.query('starts == @closest_date')
            text=data1_conf_exp['description'].astype(str).sum()
            employer.append(line['employer'].values[0])
            work_city.append(line['city'].values[0])
            work_position.append(line['position'].values[0])
            work_description.append(text)
        except Exception as e:
            print('ESCEPTION RASED:')
            continue
        
        # Выделение датасета с инфой по учебе
        try:
            par=0
            data1_conf_edu=pd.json_normalize(data1['resumes'][0][i]['educationItem'])
            for elem in data1_conf_edu['education_level'].unique():
                if elem in education_cr:
                    education.append(1)
                    par=1
                    break
            if par==0:
                education.append(0)
        except Exception as e:
            print('ESCEPTION RASED (in edu extraction for confirmed resumes):')
dataset['uuid']=pd.Series(uuid)
dataset['education']=pd.Series(education)
dataset['average_workout']=pd.Series(average_workout)
dataset['employer']=pd.Series(employer)
dataset['work_city']=pd.Series(work_city)
dataset['work_position']=pd.Series(work_position)
dataset['work_description']=pd.Series(work_description)
dataset['vacancy uuid']=str(data1['vacancy.uuid'])
dataset['vacancy name']=str(data1['vacancy.name'])
dataset['vacancy keywords']=str(data1['vacancy.keywords'])
dataset['vacancy description']=str(data1['vacancy.description'])
dataset['vacancy_comment']=str(data1['vacancy.comment'])
dataset['language']=pd.Series(language)
dataset['about']=pd.Series(about)
dataset['key_skills']=pd.Series(key_skills)
dataset.info()

: 

In [ ]:
dataset[["employer", "work_position", "work_city"]].iloc[:3]

: 

In [ ]:
dataset["work_description"].iloc[0]

: 

#### Save baseline dataset

In [ ]:
dataset.to_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\base_dataset.csv", index=False)

: 

In [ ]:
dataset = pd.read_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\base_dataset.csv")

: 

# Features processing

In [ ]:
# drop useless features
cols_to_drop = ["birth_date","first_name", "last_name", "country", "city", "work_city", "experienceItem", "educationItem", "employer"]
dataset = dataset.loc[:, ~dataset.columns.isin(cols_to_drop)]

: 

In [ ]:
# use last work position instead of null in about and key skills columns
dataset["about"] = dataset.apply(lambda x: x["work_position"] if x["about"] is None else x["about"], axis=1)
dataset["key_skills"] = dataset.apply(lambda x: x["work_position"] if x["key_skills"] is None else x["key_skills"], axis=1)

: 

In [ ]:
# fill null values of answer and education with zeros
dataset["answer"] = dataset["answer"].fillna(0)
dataset["education"] = dataset["education"].fillna(0)

: 

In [ ]:
# merge languageItem amd languageItems
dataset["languages"] = dataset.apply(lambda x: 1 if "Английский" in str(x["languageItem"]) or "Английский" in str(x["languageItems"]) else 0, axis=1)
dataset = dataset.drop(["languageItem", "languageItems"], axis=1)

: 

In [ ]:
dataset.to_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\df_not_embedding.csv", index=False)

: 

In [ ]:
dataset.head()

: 

In [ ]:
dataset.info()

: 

In [ ]:
dataset.apply(
    lambda x: (str(x["about"]) + str(x["key_skills"]) + str(x["work_position"]) + str(x["work_description"]))[:1000], axis=1
)

: 

: 

: 